In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import locale
locale.setlocale(locale.LC_ALL, 'ID')
import re

In [422]:
def getIndeksLink(links, page, cat_link, category, date=datetime.strftime(datetime.today(), '%m/%d/%Y')):
    """
    Untuk mengambil seluruh url 
    link pada indeks category tertentu
    date format : dd/mm/YYYY
    """
    if cat_link == 'news':
        url = "https://"+cat_link+".detik.com/indeks/all/"+str(page)+"?date="+date
    else :
        url = "https://"+cat_link+".detik.com/indeks/"+str(page)+"?date="+date
    # Make the request and create the response object: response
    response = requests.get(url)
    # Extract HTML texts contained in Response object: html
    html = response.text
    # Create a BeautifulSoup object from the HTML: soup
    soup = BeautifulSoup(html, "html5lib")
    contentDiv = soup.find('ul', attrs={'id':'indeks-container'})
    indeks = contentDiv.findAll('article')
    for post in indeks:
        link = [post.find('a', href=True)['href'], category]
        links.append(link)
        
    el_page = soup.find('div', class_="paging paging2")
    if el_page:
        max_page = int(soup.find('div', class_="paging paging2").findAll('a')[-2].text.replace('\n', '').strip(' '))
    
        if page <= max_page:
            links = getIndeksLink(links, i+1, cat_link, category, date)
        
    return url

In [423]:
def getDetailBerita(links):
    articles = {}
    for link in links:
        #link
        articles['url'] = link[0]
        response = requests.get(url)
        html = response.text
        # Create a BeautifulSoup object from the HTML: soup
        soup = BeautifulSoup(html, "html5lib")
        articles['id'] = int(soup.find("meta", attrs={'name':'articleid'})['content'])
        #category
        articles['category'] = link[1]
        
        #extract subcategory from breadcrumb
        bc = soup.find('div', class_="breadcrumb")
        articles['subcategory'] = bc.findAll('a')[1].text
        
        article = soup.find('article')
        
        #extract date
        pubdate = article.find('div', class_='date')
        pubdate = pubdate.text.replace('WIB', '').strip(' \t\n\r')
        articles['pubdate'] = datetime.strftime(datetime.strptime(pubdate, "%A %d %B %Y, %H:%M"), '%Y-%m-%d %H:%M')
        
        #extract author
        articles['author'] = article.find('div', class_="author").text
        
        #extract title
        articles['title'] = article.find('div', class_="jdl").find('h1').text
        
        #source
        articles['source'] = 'detik'
        
        #extract comments count
        articles['comments'] = int(soup.find('a', class_="komentar").find('span').text.replace('Komentar', '').strip(' \t\n\r'))
        
        #extract tags
        tags = article.find('div', class_="detail_tag").findAll('a')
        articles['tags'] = ','.join([x.text for x in tags])
        
        #extract images
        articles['images'] = article.find('div', class_="pic_artikel").find('img')['src']
        
        #extract detail
        detail = article.find('div', class_="detail_text")
        
        #hapus link sisip
        for link in detail.findAll('table', class_="linksisip"):
            link.decompose()
        
        #hapus video sisip
        for tag in detail.findAll('div', class_="detail_tag"):
            tag.decompose()
        
        #extract content
        detail = BeautifulSoup(detail.decode_contents().replace('<br/>', ' '), "html5lib")
        content = re.sub(r'\n|\t|\b|\r','',detail.text)
        articles['content'] = re.sub(r'(Tonton juga).*','', content)
        
        return articles

In [9]:
# links = getIndeksLink([], 1, 'news', 'news')

In [425]:
links

'https://news.detik.com/indeks/all/1?date=07/26/2018'

In [16]:
url = "https://oto.detik.com/indeks/1?date=07/26/2018"
    # Make the request and create the response object: response
response = requests.get(url)
    # Extract HTML texts contained in Response object: html
html = response.text
    # Create a BeautifulSoup object from the HTML: soup
soup = BeautifulSoup(html, "html5lib")

In [17]:
el_page = soup.find('div', class_="paging paging2")

In [391]:
if not el_page:
    print('tes')

tes


In [21]:
articles = {}
url = "https://oto.detik.com/tips-and-tricks-motor/d-4136157/lapisan-tear-off-untuk-helm-harian-perlukah"

In [22]:
response = requests.get(url)
html = response.text
        # Create a BeautifulSoup object from the HTML: soup
soup = BeautifulSoup(html, "html5lib")
articles['id'] = int(soup.find("meta", attrs={'name':'articleid'})['content'])

In [23]:
        #extract subcategory from breadcrumb
bc = soup.find('div', class_="breadcrumb")
articles['subcategory'] = bc.findAll('a')[0].text
articles['subcategory']

'detikOto'

In [24]:
article = soup.find('article')

In [25]:
article

<article class="box pd15">

                <div class="follow_btn">
FOLLOW detikOto  
<a class="fb" href="https://facebook.com/detikoto" target="_blank">
        <img alt="" src="https://cdn.detik.net.id/oto/images/icon_fb.png?v=2.9.7"/>
    </a>
    <a class="tw" href="https://twitter.com/detikoto" target="_blank">
        <img alt="" src="https://cdn.detik.net.id/oto/images/icon_tw.png?v=2.9.7"/>
    </a>
    <a class="gplus" href="https://plus.google.com/u/0/+detikcom/posts" target="_blank">
        <img alt="" src="https://cdn.detik.net.id/oto/images/icon_gplus.png?v=2.9.7"/>
    </a>
    <a class="ig" href="https://instagram.com/detikoto" target="_blank">
        <img alt="" src="https://cdn.detik.net.id/oto/images/icon_insta.png?v=2.9.7"/>
    </a>
</div>

                <!--S:BREADCRUMB-->
                <div class="breadcrumb breadcrumb_ga">
    <a href="https://oto.detik.com">detikOto</a> /
            
        <a href="https://oto.detik.com/tips-and-tricks-motor"> Tips And

In [32]:
pubdate = article.find('div', class_='date')
pubdate

<div class="date">Jumat, 27 Jul 2018 10:30 WIB</div>

In [33]:
pubdate = pubdate.text.replace('WIB', '').strip(' \t\n\r')
pubdate

'Jumat, 27 Jul 2018 10:30'

In [39]:
# articles['pubdate'] = datetime.strftime(datetime.strptime(pubdate, "%A, %d %B %Y %H:%M"), '%Y-%m-%d %H:%M')

In [42]:
articles['images'] = article.find('div', class_="pic_artikel").find('img')['src']

AttributeError: 'NoneType' object has no attribute 'find'

In [44]:
detail = article.find('div', class_="detail_text")
detail

<div class="detail_text" id="detikdetailtext">

        
        <b>Jakarta</b> - <a href="https://www.detik.com/tag/helm" target="_blank">Helm</a> ganteng saat ini banyak digunakan oleh pengendara sepeda motor. Tak cuma jadi perangkat pelindung kepala, <a href="https://www.detik.com/tag/helm" target="_blank">helm</a> saat ini juga menjadi 'obat ganteng' penggunanya.<br/><br/>Salah satu yang bikin helm tambah ganteng adalah penggunaan <a href="https://www.detik.com/tag/flat-visor" target="_blank">flat visor</a>. <a href="https://www.detik.com/tag/flat-visor" target="_blank">Flat visor</a> merupakan kaca helm datar yang biasanya digunakan di balapan, seperti pebalap MotoGP. Banyak keuntungan dari kaca helm datar itu, salah satunya memberikan visibilitas tanpa adanya distorsi.<br/><br/>Selain itu, dengan flat visor pemilik helm juga bisa menempelkan lapisan tear off. Tear off adalah sebuah lapisan khusus pada visor atau kaca helm untuk melindungi visor tersebut. Biasanya dipakai di balap